In [ ]:
!pip install -U scikit-learn

     |████████████████████████████████| 22.3MB 126kB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# cleaned!
windows = np.load('/content/drive/MyDrive/dataset/java_windows.npy')
labels = np.load('/content/drive/MyDrive/dataset/java_labels.npy')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    windows, labels, test_size=0.2, shuffle=False)

In [ ]:
del windows, labels

In [ ]:
X_test, X_val, y_test, y_val = train_test_split(
     X_test, y_test, test_size=0.5, shuffle=False)

In [ ]:
X_test.shape

(50695, 100)

In [ ]:
lw = len(X_test) + len(X_train) + len(X_val)
len(X_test)/lw, len(X_train)/lw, len(X_val)/lw

(0.10000019725850134, 0.7999996054829973, 0.10000019725850134)

In [ ]:
# check sequences order
print(''.join([chr(int(c)) for c in X_train[1]]), end='')
print(''.join([chr(int(c)) for c in X_train[2]]), end='')

{
        System.out.print(str.charAt(i - 1) + "***");
        if (i == 1) {
            i = 0;
        }
    }
    if (str.length() % 5 > 0) {
        System.out.print(str.charAt(str.length() - 1));


In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Embedding, Dropout, Bidirectional, LSTM, Dense, TimeDistributed, Flatten
from keras.metrics import AUC, Precision, Recall, SpecificityAtSensitivity, TruePositives, FalsePositives, TrueNegatives, BinaryAccuracy

n_vocab = 256 
num_lstm_units = 256 
bimodel = Sequential() 
bimodel.add(Embedding(n_vocab,20, input_length =100)) 
bimodel.add(Dropout(.2)) 
bimodel.add(Bidirectional(LSTM(num_lstm_units, return_sequences=True))) 
bimodel.add(Dropout(.2)) 
bimodel.add(TimeDistributed(Dense(150, activation='relu'))) 
bimodel.add(Dropout(.2)) 
bimodel.add(TimeDistributed(Dense(75, activation = 'relu'))) 
bimodel.add(Dropout(.2)) 
bimodel.add(TimeDistributed(Dense(1, activation = 'sigmoid'))) 
bimodel.add(Flatten())

# TAKE CARE
#bimodel = load_model('/content/drive/MyDrive/dataset/model-cleaned-31-0.1847831755876541.h5')
bimodel.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['acc', BinaryAccuracy(), Recall(), Precision(), AUC(curve='PR')])

In [ ]:
# JUST FIND F1
bimodel = load_model('/content/drive/MyDrive/dataset/model-31.h5')

# bimodel.evaluate(X_test, y_test, verbose=0)
y_pred = bimodel.predict(X_test)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_filepath = '/content/drive/MyDrive/data/best-model.h5'
#checkpoint_filepath = '/content/drive/MyDrive/dataset/model-cleaned-31-0.1847831755876541.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath, verbose=1,save_best_only=True)

In [ ]:
batch_size = 64
num_epochs = 1000

bimodel.fit(
    X_train, 
    y_train, 
    batch_size=batch_size,
    epochs=num_epochs,
    validation_data=(X_val, y_val), 
    verbose=1,
    shuffle=False,
    callbacks=[model_checkpoint_callback, EarlyStopping(monitor='val_loss', patience=20)])

Epoch 1/1000
6337/6337 [==============================] - 194s 30ms/step - loss: 0.0110 - acc: 0.2067 - binary_accuracy: 0.9974 - recall_1: 0.5817 - precision_1: 0.6480 - auc_1: 0.6300 - val_loss: 0.0015 - val_acc: 0.2739 - val_binary_accuracy: 0.9995 - val_recall_1: 0.8436 - val_precision_1: 0.9161 - val_auc_1: 0.9365

Epoch 00001: val_loss improved from inf to 0.00149, saving model to /content/drive/MyDrive/data/shuff-best-model.h5
Epoch 2/1000
6337/6337 [==============================] - 188s 30ms/step - loss: 0.0016 - acc: 0.2462 - binary_accuracy: 0.9994 - recall_1: 0.8602 - precision_1: 0.9017 - auc_1: 0.9337 - val_loss: 0.0013 - val_acc: 0.2432 - val_binary_accuracy: 0.9995 - val_recall_1: 0.8864 - val_precision_1: 0.9169 - val_auc_1: 0.9516

Epoch 00002: val_loss improved from 0.00149 to 0.00125, saving model to /content/drive/MyDrive/data/shuff-best-model.h5
Epoch 3/1000
6337/6337 [==============================] - 187s 30ms/step - loss: 0.0013 - acc: 0.2419 - binary_accuracy:

In [ ]:
scores = bimodel.evaluate(X_test, y_test, verbose=0)

In [ ]:
# ['acc', Recall(), Precision(), AUC(curve='PR')]
print("%s: %.2f%%" % (bimodel.metrics_names[5], scores[2]*100))

auc_3: 99.97%


In [ ]:
pred = bimodel.predict(X_test, batch_size=128)

In [ ]:
pred_classes = bimodel.predict_classes(X_test, batch_size=128)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
pred = np.array(pred)
y_real = y_test

In [ ]:
pred_flat = pred.flatten()
y_real_flat = y_real.flatten()

In [ ]:
pred_classes = np.array(pred_flat > 0.3, dtype='int')
pred_classes.shape

(5069500,)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_real_flat, pred_classes))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   5057341
           1       0.92      0.95      0.94     12159

    accuracy                           1.00   5069500
   macro avg       0.96      0.98      0.97   5069500
weighted avg       1.00      1.00      1.00   5069500



In [ ]:
X_test[0:10].shape

(10, 100)

In [ ]:
tp = bimodel.predict(X_test[50:200])

In [ ]:
ftp = np.array(tp > 0.5).reshape(1, tp.shape[0] * tp.shape[1])
fxt = np.array(X_test[50:200]).reshape(1, tp.shape[0] * tp.shape[1])
ftp.shape

(1, 15000)

In [ ]:
for c in range(len(fxt[0])):
  if ftp[0][c]:
    print('\n---------------------------------------\n')
  else:
    print(chr(int(fxt[0][c])), end='')

                "%d au%n"+
                  "%.1f ly%n"+
                  "%f pc",
               mate is the maximum observed value */
        double k = yvalues[ yvalues.length -1 ];
        /* Gry";
    while (choice.equalsIgnoreCase("y")) {

        System.out.println("Enter numerical grade: "

            line = reader.readLine();
            System.out.println(line);

        } catch (IOExrseY);
            }
            if (character.contains("z")) {
                System.out.println(me int activeThreads = 0;
  private long c1 = 0;
  private long c2 = 0;

  public void inc1() {
    t; responseCode++) {
            UploadResponse response = UploadResponse.getUploadResponse(responseC, int b) {
  if (a - b < 0) { // or simply (a < b) 
    return 0;
  }

  return divide(a - b, b) + 1tring[] args) {

    Properties prop = new Properties();
    OutputStream output = null;

    try {
.nextInt();  
 System.out.print("Enter lengtht: ");
 int length = scanner.nextInt();
 this.width = w

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test.flatten(), y_pred.flatten() > .5))